# Modelo de traducción de texto
El modelo de phishing parece funcionar tanto en español como en inglés, pero por razones de confiabilidad, utilizaremos un modelo traductor primero.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, pipeline
from torch import argmax, softmax
# Modelo de traducción
tokenizer_traductor = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_traductor = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

c:\Código\GitHub\modelo_traduccion_fishing\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def traductor(text):
    input = tokenizer_traductor(text, return_tensors="pt", padding=True)
    output = model_traductor.generate(**input)
    traduccion = tokenizer_traductor.batch_decode(output, skip_special_tokens=True)
    return traduccion

In [3]:
# Reemplazar con la conexión a base de datos
lista_textos = [
    "Hola Mundo",
    "Si no verificas tu cuenta en las próximas 24 horas, podríamos vernos obligados a imponer restricciones adicionales o suspensor tu cuenta de forma permanente por motivos de seguridad. Proteger tu información es nuestra prioridad. Si no fue tú quien realizó esta actividad, te recomendamos que cambie tu contraseña inmediatamente después de verificar tu identidad a través del enlace propuesto.",
    "No quiero, hazlo tu"
]

traducciones_lista = traductor(lista_textos)
print(traducciones_lista)

['Hello, World.', 'If you do not check your account within the next 24 hours, we may be forced to impose additional restrictions or suspend your account permanently for security reasons. Protecting your information is our priority. If it was not you who did this activity, we recommend that you change your password immediately after verifying your identity through the proposed link.', "I don't want to, you do."]


# Modelo de detección de phishing

In [ ]:
# Modelo de phishing
tokenizer_phishing = AutoTokenizer.from_pretrained("ealvaradob/bert-finetuned-phishing")
model_phishing = AutoModelForSequenceClassification.from_pretrained("ealvaradob/bert-finetuned-phishing")

In [ ]:
def phishing(text):
    input = tokenizer_phishing(text, return_tensors="pt", padding=True)
    output = model_phishing(**input)
    logits = output.logits
    probabilidades = logits.softmax(dim=1)
    prediccion_id = argmax(probabilidades, dim=1)
    lista_resultados = []
    
    for i in range(len(prediccion_id)):
        pred_id = prediccion_id[i].item()
        etiqueta_predicha = model_phishing.config.id2label[pred_id]
        prob_predicha = probabilidades[i, pred_id].item()

        resultado = {   
            "prediccion" : "phishing" if etiqueta_predicha == "phishing" else "benigno",
            "probabilidad" : prob_predicha
        }
        lista_resultados.append(resultado)
    return lista_resultados

In [ ]:
def prediccion(text):
    traduccion = traductor(text)
    resultado = phishing(traduccion)
    return resultado

In [ ]:
def correo(text):
    resultado = phishing(text)
    return resultado

In [7]:
resultado = prediccion(lista_textos)
print(resultado)

[{'prediccion': 'benigno', 'probabilidad': 0.9975121021270752}, {'prediccion': 'phishing', 'probabilidad': 0.9996534585952759}, {'prediccion': 'benigno', 'probabilidad': 0.9999877214431763}]
